# ナイーブベイズ分類器による文書分類

In [ ]:
from prettyprint import pp

## 1. 形態素解析

In [ ]:
import MeCab
mecab = MeCab.Tagger('mecabrc')
text = "はじめに こんにちは。システム開発部の中村です。 機械学習についての理解を促進するため、 データから分類モデルを自動で構築する古典的な方法である"
node = mecab.parseToNode(text)
# while node:
#     if node.feature.split(',')[0] == '名詞':
#         print node.surface.lower()
#     node = node.next
    
def get_words_list(text):
    mecab = MeCab.Tagger('mecabrc')
    node = mecab.parseToNode(text)
    words_list = []
    while node:
        if node.feature.split(",")[0] == '名詞':
            words_list.append(node.surface.lower())
        node = node.next
    return words_list

def get_words_matrix(texts_list):
    """
    texts_list : ['text1', 'text2',... ]
    """
    res = []
    return [get_words_list(text) for text in texts_list]

### mecabの例

In [ ]:
text1 = """
物語は、女性ファッション誌の編集者になることを夢見る女性・河野悦子が、総合出版社・景凡社の校閲部の社員として採用されるというもの。「自分のいるべき場所はここじゃない」と思いつつも、仕事にまい進する校閲者・悦子の“地味にスゴイ”活躍を描く。
物語に入っていくための丁寧な導入と、細かな“リアルさ”をぶっ飛ばす石原さとみというキャスティングの妙が、『校閲ガール』の好評ぶりにつながっているのだろう。
自動運転車の開発競争は世界中で加熱しつつある。
"""
text2 = """
オクスフォード大学からのスピンオフにより生まれた自動運転車のスタートアップであるOxboticaが、自動運転技術を搭載したルノーにてイギリスでの公道実験を開始した。イギリス公道上を自動運転車が走るのは初めてのことだ。場所はイングランドのミルトン・キーンズで、走行速度は時速5マイルだとのこと。まだ試験段階ではあるものの、自動運転車は広がりつつある
"""
res = get_words_matrix([text1,text2])
print str(res).decode('string-escape')

## 2. 特徴ベクトルの作成

In [ ]:
from gensim import corpora, matutils

import numpy as np


dictionary = corpora.Dictionary(res)


bows = dictionary.doc2bow(res[0]) # [(w_id1, w_id1_cnt), (w_id2, w_id2_cnt),...]
bows = [dictionary.doc2bow(x) for x in res]
# print '----- BoW -------'
# print bows
# print 
train_X = np.array([(matutils.corpus2dense([vec], num_terms=len(dictionary)).T[0]) 
                   for vec in bows])
train_y = np.array([1,0])
# print '------- 特徴ベクトル --------'
# print feature_vectors


## 3. ナイーブベイズ分類器

In [ ]:
# sklearnで動作確認
from sklearn.naive_bayes import MultinomialNB

### randomな例（ラベル種類数 : 3、特徴ベクトル次元数 : 200）

In [ ]:
random_train_X = np.random.randint(4, size=(100,200))#cntは最大4とする
random_train_Y = np.random.randint(3,size=100) # 正解ラベル数は3
clf = MultinomialNB(alpha=0.1)
clf.fit(random_train_X,random_train_Y)
random_test_X = np.random.randint(3,size=(5,200))
print clf.predict(random_test_X)
print clf.predict_proba(random_test_X)